In [59]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

Leemos los datos descargados con ayuda del API de idealista, al costado de cada csv se indica el mes de descarga de cada Dataframe.

In [60]:
path='C:/Users/nicol/OneDrive/Documentos/VSCLocal/Data/TFM/'
df_02=pd.read_csv(path+"idealista_02.csv")
df_03=pd.read_csv(path+"idealista_03.csv")
df_04=pd.read_csv(path+"idealista_04.csv")

Agregamos una columna de mes a cada DataFrame y eliminamos la columna 'priceInfo'

In [61]:
df_02['Month']='Febrero'
df_03['Month']='Marzo'
df_04['Month']='Abril'

df_03.drop('priceInfo', axis=1, inplace=True)
df_04.drop('priceInfo', axis=1, inplace=True)

Verificamos que cada Dataframe tiene la misma estructura    

In [62]:
print(df_02.shape)
print(df_03.shape)
print(df_04.shape)

(5000, 43)
(4750, 43)
(4650, 43)


Unimos los Dataframes y eliminanos los duplicados en función de la columna 'propertyCode'manteniendo la última oferta de idealista 

In [63]:
# Unir los DataFrames en una lista
dfs = [df_02,df_03,df_04]

# Concatenar los DataFrames
merged_df = pd.concat(dfs,ignore_index=True)

# Eliminar filas duplicadas basadas en la columna 'propertyCode', manteniendo la última aparición
merged_df = merged_df.drop_duplicates(subset='propertyCode', keep='last')

merged_df.shape

(11680, 43)

Hacemos un groub by para ver la distribución de las propiedades por municipio

In [64]:
merged_df.groupby('municipality').count()['province']

municipality
Getafe                    2
Leganés                  63
Madrid                11314
Pozuelo de Alarcón      301
Name: province, dtype: int64

Filtramos el Dataframe para tener solo las propiedades dentro de la municipalidad de Madrid, y las variables a trabajar. Finalmente eliminamos cualquier oferta que tenga un NaN.

In [65]:
filtered_df=merged_df[merged_df['municipality']=='Madrid']

variables=['price', 'floor', 'propertyType', 'size', 'exterior', 'rooms', 'bathrooms','latitude', 'longitude', 'distance',
           'status', 'newDevelopment', 'hasLift','priceByArea', 'parkingSpace', 'newDevelopmentFinished']

filtered_df=filtered_df[variables]

filtered_df


,price,floor,propertyType,size,exterior,rooms,bathrooms,latitude,longitude,distance,status,newDevelopment,hasLift,priceByArea,parkingSpace,newDevelopmentFinished
0,1197000.0,2,flat,217.0,True,4,3,40.419531,-3.705679,375,renew,False,True,5516.0,NaN,NaN
1,91000.0,bj,flat,85.0,True,3,2,40.343316,-3.692217,8212,renew,False,False,1071.0,NaN,NaN
2,279000.0,3,flat,129.0,True,2,2,40.473896,-3.708138,6373,good,False,True,2163.0,NaN,NaN
4,350000.0,1,flat,93.0,True,3,1,40.401887,-3.694255,1814,good,False,True,3763.0,NaN,NaN
5,70700.0,4,flat,113.0,True,3,2,40.381567,-3.654923,5657,good,False,True,626.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14395,15000000.0,7,flat,945.0,True,6,6,40.418520,-3.686352,1444,renew,False,True,15873.0,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN
14396,3450000.0,3,flat,270.0,True,3,4,40.425852,-3.683221,1977,good,False,True,12778.0,NaN,NaN
14397,2750000.0,3,flat,190.0,True,3,4,40.419988,-3.690723,1121,good,False,True,14474.0,NaN,NaN
14398,3900000.0,3,flat,240.0,True,4,5,40.424490,-3.686366,1671,good,False,True,16250.0,NaN,NaN


In [66]:
# Vemos los tipos de datos para cada columna si es númerico, bool u object
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11314 entries, 0 to 14399
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   price                   11314 non-null  float64
 1   floor                   10340 non-null  object 
 2   propertyType            11314 non-null  object 
 3   size                    11314 non-null  float64
 4   exterior                10198 non-null  object 
 5   rooms                   11314 non-null  int64  
 6   bathrooms               11314 non-null  int64  
 7   latitude                11314 non-null  float64
 8   longitude               11314 non-null  float64
 9   distance                11314 non-null  int64  
 10  status                  11314 non-null  object 
 11  newDevelopment          11314 non-null  bool   
 12  hasLift                 10719 non-null  object 
 13  priceByArea             11314 non-null  float64
 14  parkingSpace            3056 non-null   obj

Corregimos los datos de la columna floor

In [67]:
# Reemplazar 'bj' por '0' en la columna 'floor'
filtered_df['floor'] = filtered_df['floor'].replace(['bj','en'], 0)

# Reemplazar 'ss' por '-1' en la columna 'floor'
filtered_df['floor'] = filtered_df['floor'].replace('ss', '-1')

# Reemplazar 'st' por 'la moda' en la columna 'floor' que es 1
filtered_df['floor'] = filtered_df['floor'].replace('st', '1')

# Reemplazar 'NaN' por 'la moda' en la columna 'floor' que es 1
filtered_df.fillna({'floor': '1'}, inplace=True)

# Realizar la agrupación y contar el número de propiedades de cada tipo
grouped_counts = filtered_df.groupby('floor').count()['propertyType']

# Ordenar el DataFrame resultante por el número de propiedades de cada tipo
sorted_counts = grouped_counts.sort_values(ascending=False)

# Convertir la columna 'floor' de tipo 'object' a tipo 'integer' 
filtered_df['floor'] = filtered_df['floor'].astype(int)

# Mostrar el resultado
filtered_df['floor'].unique()

array([ 2,  0,  3,  1,  4, -1,  5,  7,  8, 13,  6, 10,  9, -2, 18, 12, 22,
       11, 15, 14, 19, 17, 20, 21, 24, 16])

Corregimos los datos de la columna exterior            

In [68]:
# Reemplazar 'Nan' por 'False' en la columna 'exterior'
filtered_df.fillna({'exterior': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'exterior'
filtered_df['exterior'] = filtered_df['exterior'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'exterior'
filtered_df['exterior'] = filtered_df['exterior'].replace('False', 0)

# Convertir la columna 'exterior' de tipo 'object' a tipo 'integer' 
filtered_df['exterior'] = filtered_df['exterior'].astype(int)

# Mostrar el resultado
filtered_df['exterior'].unique()

# Re

array([1, 0])

Corregimos los datos de la columna newDevelopment

In [69]:
# Reemplazar 'True' por '1' en la columna 'newDevelopment'
filtered_df['newDevelopment'] = filtered_df['newDevelopment'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'newDevelopment'
filtered_df['newDevelopment'] = filtered_df['newDevelopment'].replace('False', 0)

# Convertir la columna 'newDevelopment' de tipo 'object' a tipo 'integer' 
filtered_df['newDevelopment'] = filtered_df['newDevelopment'].astype(int)

# Mostrar el resultado
filtered_df['newDevelopment'].unique()

array([0, 1])

Corregimos los datos de la columna hasLift

In [70]:
# Reemplazar 'Nan' por 'False' en la columna 'hasLift'
filtered_df.fillna({'hasLift': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'hasLift'
filtered_df['hasLift'] = filtered_df['hasLift'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'hasLift'
filtered_df['hasLift'] = filtered_df['hasLift'].replace('False', 0)

# Convertir la columna 'hasLift' de tipo 'object' a tipo 'integer' 
filtered_df['hasLift'] = filtered_df['hasLift'].astype(int)

# Mostrar el resultado
filtered_df['hasLift'].unique()

array([1, 0])

Corregimos los datos de la columna newDevelopmentFinished

In [71]:
# Reemplazar 'Nan' por 'False' en la columna 'newDevelopmentFinished'
filtered_df.fillna({'newDevelopmentFinished': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'newDevelopmentFinished'
filtered_df['newDevelopmentFinished'] = filtered_df['newDevelopmentFinished'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'newDevelopmentFinished'
filtered_df['newDevelopmentFinished'] = filtered_df['newDevelopmentFinished'].replace('False', 0)

# Convertir la columna 'newDevelopmentFinished' de tipo 'object' a tipo 'integer' 
filtered_df['newDevelopmentFinished'] = filtered_df['newDevelopmentFinished'].astype(int)

# Mostrar el resultado
filtered_df['newDevelopmentFinished'].unique()

array([0, 1])

Corregimos los datos de la columna parkingSpace y generamos tres columnas : hasParkingSpace , isParkingSpaceIncludedInPrice , parkingSpacePrice

In [74]:
import pandas as pd
import ast

# Función para procesar cada fila
def process_row(row):
    if pd.isna(row):
        return pd.Series({'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 0})
    else:
        try:
            row_dict = ast.literal_eval(row)  # Convertir la cadena a un diccionario
            has_parking_space = row_dict.get('hasParkingSpace', False)
            is_included = row_dict.get('isParkingSpaceIncludedInPrice', False)
            price = row_dict.get('parkingSpacePrice', 0) if is_included else 0
        except (SyntaxError, ValueError):  # Manejar el caso de que la cadena no sea un diccionario válido
            has_parking_space = False
            is_included = False
            price = 0
        return pd.Series({'hasParkingSpace': has_parking_space, 'isParkingSpaceIncludedInPrice': is_included, 'parkingSpacePrice': price})

# Aplicar la función a cada fila
new_columns = filtered_df['parkingSpace'].apply(process_row)

# Concatenar los resultados al DataFrame original
filtered_df = pd.concat([filtered_df, new_columns], axis=1)

filtered_df


,price,floor,propertyType,size,exterior,rooms,bathrooms,latitude,longitude,distance,status,newDevelopment,hasLift,priceByArea,parkingSpace,newDevelopmentFinished,hasParkingSpace,isParkingSpaceIncludedInPrice,parkingSpacePrice
0,1197000.0,2,flat,217.0,1,4,3,40.419531,-3.705679,375,renew,0,1,5516.0,NaN,0,False,False,0
1,91000.0,0,flat,85.0,1,3,2,40.343316,-3.692217,8212,renew,0,0,1071.0,NaN,0,False,False,0
2,279000.0,3,flat,129.0,1,2,2,40.473896,-3.708138,6373,good,0,1,2163.0,NaN,0,False,False,0
4,350000.0,1,flat,93.0,1,3,1,40.401887,-3.694255,1814,good,0,1,3763.0,NaN,0,False,False,0
5,70700.0,4,flat,113.0,1,3,2,40.381567,-3.654923,5657,good,0,1,626.0,NaN,0,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14395,15000000.0,7,flat,945.0,1,6,6,40.418520,-3.686352,1444,renew,0,1,15873.0,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",0,True,True,0
14396,3450000.0,3,flat,270.0,1,3,4,40.425852,-3.683221,1977,good,0,1,12778.0,NaN,0,False,False,0
14397,2750000.0,3,flat,190.0,1,3,4,40.419988,-3.690723,1121,good,0,1,14474.0,NaN,0,False,False,0
14398,3900000.0,3,flat,240.0,1,4,5,40.424490,-3.686366,1671,good,0,1,16250.0,NaN,0,False,False,0


Corregimos los datos de la columna hasParkingSpace

In [75]:
# Reemplazar 'Nan' por 'False' en la columna 'hasParkingSpace'
filtered_df.fillna({'hasParkingSpace': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'hasParkingSpace'
filtered_df['hasParkingSpace'] = filtered_df['hasParkingSpace'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'hasParkingSpace'
filtered_df['hasParkingSpace'] = filtered_df['hasParkingSpace'].replace('False', 0)

# Convertir la columna 'hasParkingSpace' de tipo 'object' a tipo 'integer' 
filtered_df['hasParkingSpace'] = filtered_df['hasParkingSpace'].astype(int)

# Mostrar el resultado
filtered_df['hasParkingSpace'].unique()

array([0, 1])

Corregimos los datos de la columna isParkingSpaceIncludedInPrice

In [76]:
# Reemplazar 'Nan' por 'False' en la columna 'isParkingSpaceIncludedInPrice'
filtered_df.fillna({'isParkingSpaceIncludedInPrice': 'False'}, inplace=True)

# Reemplazar 'True' por '1' en la columna 'isParkingSpaceIncludedInPrice'
filtered_df['isParkingSpaceIncludedInPrice'] = filtered_df['isParkingSpaceIncludedInPrice'].replace('True', 1)

# Reemplazar 'False' por '0' en la columna 'isParkingSpaceIncludedInPrice'
filtered_df['isParkingSpaceIncludedInPrice'] = filtered_df['isParkingSpaceIncludedInPrice'].replace('False', 0)

# Convertir la columna 'isParkingSpaceIncludedInPrice' de tipo 'object' a tipo 'integer' 
filtered_df['isParkingSpaceIncludedInPrice'] = filtered_df['isParkingSpaceIncludedInPrice'].astype(int)

# Mostrar el resultado
filtered_df['isParkingSpaceIncludedInPrice'].unique()

array([0, 1])

Verificamos que los datos númericos y categoricos estén correctamente.

In [77]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11314 entries, 0 to 14399
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   price                          11314 non-null  float64
 1   floor                          11314 non-null  int32  
 2   propertyType                   11314 non-null  object 
 3   size                           11314 non-null  float64
 4   exterior                       11314 non-null  int32  
 5   rooms                          11314 non-null  int64  
 6   bathrooms                      11314 non-null  int64  
 7   latitude                       11314 non-null  float64
 8   longitude                      11314 non-null  float64
 9   distance                       11314 non-null  int64  
 10  status                         11314 non-null  object 
 11  newDevelopment                 11314 non-null  int32  
 12  hasLift                        11314 non-null  int3

Guardamos el Dataframe procesado en un archivo csv

In [78]:
#Definimos el nombre del archivo
nombre_cvs='datos_procesados_3.csv'

# Definimos la ruta
ruta='C:/Users/nicol/OneDrive/Documentos/VSCLocal/Data/TFM/'

# Ruta donde guardar el archivo CSV
ruta_csv = ruta+nombre_cvs

# Guardar el DataFrame en un archivo CSV
filtered_df.to_csv(ruta_csv, index=False)